In [ ]:
import os
def jpeg_compression(scale, src, dst, config):

    dim = config["n_features_per_level"]
    n_levels = config["n_levels"]

    for d in range(dim):
        for i in range(n_levels):
            src_path = os.path.join(src, f"dim{d}", f"{str(i).zfill(2)}.png")
            save_path = os.path.join(dst, f"dim{d}", str(scale).zfill(3))
            os.makedirs(save_path, exist_ok=True)
            save_path = os.path.join(save_path,  f"{str(i).zfill(2)}.jpg")

            if os.path.isfile(save_path):
                !rm $save_path
            !ffmpeg -hide_banner -i $src_path -qscale:v $scale $save_path


# keyint=7:min-keyint=7:no-scenecut:me=full:subme=7:bframes=0
def hevc_compression(crf, framerate, src, dst, config):

    dim = config["n_features_per_level"]
    
    for d in range(dim):
        src_path = os.path.join(src, f"dim{d}", "%05d.png")
        os.makedirs(os.path.join(dst, f"dim{d}"), exist_ok=True)
        save_path = os.path.join(dst, f"dim{d}", f"{crf}_{framerate}.mp4")

        if os.path.isfile(save_path):
            !rm $save_path

        !ffmpeg -framerate $framerate -i $src_path -c:v hevc -preset slow -x265-params bframes=0 -crf $crf $save_path



Compress Learnable Keyframes and Sparse Grid

In [ ]:
import json
datasets = ["jockey"]

for data in datasets:
    experiment_name = f"{data}"
    config_path = f"./logs_nvp/{experiment_name}/config.json"
    base_path = f"./logs_nvp/{experiment_name}/compression"


    with open(config_path, 'r') as f:
        config = json.load(f)

    config = config["nvp"]
    keyframe_path = os.path.join(base_path, "src", "keyframes", "xy")
    save_path = os.path.join(base_path, "dst", "keyframes", "xy")
    jpeg_compression(scale=2, src=keyframe_path, dst=save_path, config=config["2d_encoding_xy"])

    keyframe_path = os.path.join(base_path, "src", "keyframes", "xt")
    save_path = os.path.join(base_path, "dst", "keyframes", "xt")
    jpeg_compression(scale=3, src=keyframe_path, dst=save_path, config=config["2d_encoding_xt"])

    keyframe_path = os.path.join(base_path, "src", "keyframes", "yt")
    save_path = os.path.join(base_path, "dst", "keyframes", "yt")
    jpeg_compression(scale=3, src=keyframe_path, dst=save_path, config=config["2d_encoding_yt"])

    sparsegrid_path = os.path.join(base_path, "src", "sparsegrid")
    save_path = os.path.join(base_path, "dst", "sparsegrid")
    hevc_compression(crf=21, framerate=25, src=sparsegrid_path, dst=save_path, config=config["3d_encoding"])
    